In [14]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import logging

In [15]:
if os.getenv('USERNAME')    =='ywang':
    BOX_dir                 = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim 1.5\\PBA50'.format(os.getenv('USERNAME'))
    
    # input file locations
    hybrid_plu_boc_dir      = os.path.join(BOX_dir, 'Policies\\Base zoning\\outputs\\hybrid_base_zoning')
    other_inputs_dir        = os.path.join(BOX_dir, 'Policies\\Base zoning\\inputs')
    
    # output file location
    data_output_dir         = os.path.join(BOX_dir, 'Policies\\Base zoning\\outputs\\capacity')

    
ALLOWED_BUILDING_TYPE_CODES = ["HS","HT","HM","OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]
RES_BUILDING_TYPE_CODES     = ["HS","HT","HM",                                        "MR"          ]
NONRES_BUILDING_TYPE_CODES  = [               "OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]

# used in calculate_capacity()
SQUARE_FEET_PER_ACRE                = 43560.0
SQUARE_FEET_PER_DU                  = 1200.0
FEET_PER_STORY                      = 11.0
PARCEL_USE_EFFICIENCY               = 0.5
SQUARE_FEET_PER_EMPLOYEE            = 350.0
SQUARE_FEET_PER_EMPLOYEE_OFFICE     = 175.0
SQUARE_FEET_PER_EMPLOYEE_INDUSTRIAL = 500.0

## export data // will visualize in Tableau
today = datetime.today().strftime('%Y_%m_%d')

## Precessed PLU BOC data
## four versions of hybrid zoning for BASIS; refers to different versions of the hybrid plu data
versions = ['_fill_naType','_BASIS_intensity_all','_BASIS_intensity_partial','_BASIS_devType_intensity_partial']  
version = versions[3]

In [3]:
## Hybrid base zoning data

p10_plu_boc = pd.read_csv(os.path.join(hybrid_plu_boc_dir, today+'_p10_plu_boc' + version + '.csv'))

print("p10_plu_boc.county_id.value_counts()")
display(p10_plu_boc.county_id.value_counts())

# nodev_zmod value counts
print("p10_plu_boc.nodev_zmod.value_counts()")
display(p10_plu_boc.nodev_zmod.value_counts())

display(p10_plu_boc.dtypes)

p10_plu_boc.county_id.value_counts()


85    436449
1     389889
13    325202
81    195486
97    171997
75    153355
95    137961
41     96816
55     49053
Name: county_id, dtype: int64

p10_plu_boc.nodev_zmod.value_counts()


0    1893087
1      63121
Name: nodev_zmod, dtype: int64

PARCEL_ID         float64
county_id           int64
county_name        object
juris_zmod         object
ACRES             float64
                   ...   
MT_idx             object
ME_idx             object
MAX_DUA_idx        object
MAX_FAR_idx        object
MAX_HEIGHT_idx     object
Length: 72, dtype: object

In [4]:
# Assign allow residential and/or non-residential by summing the columns
# for the residential/nonresidential allowed building type codes
# Returns dataframe with PARCEL_ID, allow_res_[boc_source], allow_nonres_[boc_source]
def set_allow_dev_type(df_original,boc_source):
    # don't modify passed df
    df = df_original.copy()

    # note that they can't be null because then they won't sum -- so make a copy and fillna with 0
    for dev_type in ALLOWED_BUILDING_TYPE_CODES:
        df[dev_type+"_"+boc_source] = df[dev_type+"_"+boc_source].fillna(value=0.0)    
    
    # allow_res is sum of allowed building types that are residential
    res_allowed_columns = [btype+'_'+boc_source for btype in RES_BUILDING_TYPE_CODES]
    df['allow_res_' +boc_source] = df[res_allowed_columns].sum(axis=1)
    
    # allow_nonres is the sum of allowed building types that are non-residential
    nonres_allowed_columns = [btype+'_'+boc_source for btype in NONRES_BUILDING_TYPE_CODES]
    df['allow_nonres_'+boc_source] = df[nonres_allowed_columns].sum(axis=1)
    
    return df[['PARCEL_ID',
               "allow_res_"    +boc_source,
               "allow_nonres_" +boc_source]]

In [6]:
# Calculate capacity

def calculate_capacity(df_original,boc_source,nodev_source):
    
    df = df_original.copy()
    
    # DUA calculations apply to parcels 'allowRes' and not marked as "nodev"
    df['units_'+boc_source] = df['ACRES'] * df['max_dua_'+boc_source]   
    
    # zero out units for 'nodev' parcels or parcels that don't allow residential
    zero_unit_idx = (df['allow_res_'+boc_source] == 0) | (df['nodev_'+nodev_source] == 1)
    df.loc[zero_unit_idx,'units_'   +boc_source] = 0
        
    # FAR calculations apply to parcels 'allowNonRes' and not marked as "nodev"
    df['sqft_' +boc_source] = df['ACRES'] * df['max_far_'+boc_source] * SQUARE_FEET_PER_ACRE 
    
    # zero out sqft for 'nodev' parcels or parcels that don't allow non-residential
    zero_sqft_idx = (df['allow_nonres_'+boc_source] == 0) | (df['nodev_'+nodev_source] == 1)
    df.loc[zero_sqft_idx,'sqft_'       +boc_source] = 0
    
    df['Ksqft_'+boc_source] = df['sqft_'+boc_source]*0.001

    # of nonresidential uses, only office allowed
    office_idx   = (df['OF_'+boc_source] == 1) & (df['allow_nonres_'+boc_source]== 1)
    # of nonresidential uses, only industrial allowed
    allow_indust = df[['IL_'+boc_source,'IW_'+boc_source,'IH_'+boc_source]].sum(axis = 1)
    indust_idx   = (allow_indust > 0) & (df['allow_nonres_'+boc_source] == allow_indust)
    # calculate non-residential capacity in employment
    df[               'emp_'+boc_source] = df['sqft_'+boc_source] / SQUARE_FEET_PER_EMPLOYEE
    df.loc[office_idx,'emp_'+boc_source] = df['sqft_'+boc_source] / SQUARE_FEET_PER_EMPLOYEE_OFFICE
    df.loc[indust_idx,'emp_'+boc_source] = df['sqft_'+boc_source] / SQUARE_FEET_PER_EMPLOYEE_INDUSTRIAL
    
    if ('source_dua_' +boc_source in df.columns) & ('source_far_'+boc_source in df.columns):
        return df[['PARCEL_ID',
                   "source_dua_"   +boc_source,
                   "allow_res_"    +boc_source,
                   "units_"        +boc_source,
                   "allow_nonres_" +boc_source,
                   "source_far_"   +boc_source,
                   "sqft_"         +boc_source,
                   "Ksqft_"        +boc_source,
                   "emp_"          +boc_source]]
    else:
        return df[['PARCEL_ID', 
                   "allow_res_"    +boc_source,
                   "units_"        +boc_source,
                   "allow_nonres_" +boc_source,
                   "sqft_"         +boc_source,
                   "Ksqft_"        +boc_source,
                   "emp_"          +boc_source]]

In [7]:
# Add basis and pba40 allowed_res_ and allowed_nonres_
allowed_basis = set_allow_dev_type(p10_plu_boc, "basis")
allowed_pba40 = set_allow_dev_type(p10_plu_boc, "pba40")

p10_plu_boc.drop(columns = [
    'allow_res_basis','allow_res_pba40','allow_nonres_basis','allow_nonres_pba40'], inplace = True)


p10_plu_boc = pd.merge(left=p10_plu_boc,
                       right=allowed_basis,
                       how="left", on="PARCEL_ID")
p10_plu_boc = pd.merge(left=p10_plu_boc,
                       right=allowed_pba40,
                       how="left", on="PARCEL_ID")

['PARCEL_ID', 'county_id', 'county_name', 'juris_zmod', 'ACRES', 'pba50zoningmodcat_zmod', 'max_far_basis', 'max_far_pba40', 'source_far_basis', 'source_far_pba40', 'max_dua_basis', 'max_dua_pba40', 'source_dua_basis', 'source_dua_pba40', 'max_height_basis', 'max_height_pba40', 'nodev_zmod', 'nodev_pba40', 'building_types_source_basis', 'source_basis', 'plu_id_basis', 'plu_jurisdiction_basis', 'plu_description_basis', 'HS_basis', 'HS_pba40', 'HT_basis', 'HT_pba40', 'HM_basis', 'HM_pba40', 'OF_basis', 'OF_pba40', 'HO_basis', 'HO_pba40', 'SC_basis', 'SC_pba40', 'IL_basis', 'IL_pba40', 'IW_basis', 'IW_pba40', 'IH_basis', 'IH_pba40', 'RS_basis', 'RS_pba40', 'RB_basis', 'RB_pba40', 'MR_basis', 'MR_pba40', 'MT_basis', 'MT_pba40', 'ME_basis', 'ME_pba40', 'HS_idx', 'HT_idx', 'HM_idx', 'OF_idx', 'HO_idx', 'SC_idx', 'IL_idx', 'IW_idx', 'IH_idx', 'RS_idx', 'RB_idx', 'MR_idx', 'MT_idx', 'ME_idx', 'MAX_DUA_idx', 'MAX_FAR_idx', 'MAX_HEIGHT_idx', 'allow_res_basis', 'allow_nonres_basis', 'allow_res_pb

In [8]:
## Calculate development capacity

capacity_basis_allAtts = calculate_capacity(p10_plu_boc,'basis','zmod')
capacity_pba40_allAtts = calculate_capacity(p10_plu_boc,'pba40','zmod')

print("capacity_pba40_allAtts has {:,} rows; head:".format(len(capacity_pba40_allAtts)))
display(capacity_pba40_allAtts.head())

print("capacity_basis_allAtts has {:,} rows; head:".format(len(capacity_basis_allAtts)))
display(capacity_basis_allAtts.head())

In [10]:
# output all attributes

capacity_allAtts = pd.merge(left=capacity_pba40_allAtts, 
                            right=capacity_basis_allAtts, 
                            how="inner", 
                            on=['PARCEL_ID'])

p10_plu_boc_simply = p10_plu_boc[['PARCEL_ID','ACRES','county_id', 'county_name','juris_zmod', 'nodev_zmod'] + [
                     dev_type+'_pba40' for dev_type in ALLOWED_BUILDING_TYPE_CODES] + [
                     dev_type+'_basis' for dev_type in ALLOWED_BUILDING_TYPE_CODES] + [
                     'building_types_source_basis','source_basis','plu_id_basis', 
                     'plu_jurisdiction_basis', 'plu_description_basis']]

capacity_allAtts = capacity_allAtts.merge(p10_plu_boc_simply,
                                          on = 'PARCEL_ID', 
                                          how = 'inner')
print("capacity has {:,} rows; head:".format(len(capacity_allAtts)))
display(capacity_allAtts.head())

for i in ['PARCEL_ID', 'nodev_zmod',
          'allow_res_pba40', 'allow_res_basis','allow_nonres_pba40','allow_nonres_basis'] + [
          dev_type+'_pba40' for dev_type in ALLOWED_BUILDING_TYPE_CODES] + [
          dev_type+'_basis' for dev_type in ALLOWED_BUILDING_TYPE_CODES]:
    capacity_allAtts[i] = capacity_allAtts[i].fillna(-1).astype(np.int64)

print(capacity_allAtts.dtypes)

capacity_allAtts.to_csv(os.path.join(data_output_dir, today+'_devCapacity_allAttrs'+ version + '.csv'), index = False)

capacity has 1,956,208 rows; head:


,PARCEL_ID,source_dua_pba40,allow_res_pba40,units_pba40,allow_nonres_pba40,source_far_pba40,sqft_pba40,Ksqft_pba40,emp_pba40,source_dua_basis,...,RS_basis,RB_basis,MR_basis,MT_basis,ME_basis,building_types_source_basis,source_basis,plu_id_basis,plu_jurisdiction_basis,plu_description_basis
0,229116.0,pba40,1.0,6.721041,0.0,missing,0.0,0.0,0.0,basis,...,0.0,0.0,0.0,0.0,0.0,2010 Aksel Geo-matching,NaN,fc5f982a-40e3-452f-a660-f5f80b7d0b24,Livermore,Planned Development
1,244166.0,pba40,1.0,3.883268,0.0,missing,0.0,0.0,0.0,basis,...,0.0,0.0,0.0,0.0,0.0,2010 Aksel Geo-matching,NaN,4d5ddd2e-464a-4861-9c1f-88941a0b5676,Livermore,Residential
2,202378.0,pba40,2.0,130.444362,0.0,imputed from max_height,0.0,0.0,0.0,basis,...,0.0,0.0,0.0,0.0,0.0,2010 Aksel Geo-matching,NaN,5a6d3363-b377-4431-8ac8-35aa1593225d,Hayward,Planned Development
3,2004420.0,pba40,1.0,1.318751,0.0,imputed from max_height,0.0,0.0,0.0,basis,...,0.0,0.0,0.0,0.0,0.0,2010 Aksel Geo-matching,NaN,0b967b71-e05b-46db-a689-4d3460b22887,Unincorporated Sonoma,Land Extensive Agriculture
4,340332.0,pba40,3.0,0.000000,1.0,imputed from max_height,0.0,0.0,0.0,basis,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,9b689549-939f-4288-aad7-d3584fef122f,Fremont,Planned District


PARCEL_ID                        int64
source_dua_pba40                object
allow_res_pba40                  int64
units_pba40                    float64
allow_nonres_pba40               int64
source_far_pba40                object
sqft_pba40                     float64
Ksqft_pba40                    float64
emp_pba40                      float64
source_dua_basis                object
allow_res_basis                  int64
units_basis                    float64
allow_nonres_basis               int64
source_far_basis                object
sqft_basis                     float64
Ksqft_basis                    float64
emp_basis                      float64
ACRES                          float64
county_id                        int64
county_name                     object
juris_zmod                      object
nodev_zmod                       int64
HS_pba40                         int64
HT_pba40                         int64
HM_pba40                         int64
OF_pba40                 